In [1]:
# IMPORTS
import os
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.10f}'.format)

In [2]:
# Load files
log = pd.read_excel('C:/Users/alexb/OneDrive/DP3/VAS_Log.xlsx')
log.rename(columns = {'Monitor ID':'ID', 'Location/participant':'PPN', 'Downoad data':'download', 'Deployment date': 'deploy', 'Return date':'return'}, inplace = True)
log = log[['ID', 'PPN', 'deploy', 'return']]
log

,ID,PPN,deploy,return
0,2c564e,DP3-0001,2021-02-25 00:00:00,2021-06-10
1,488810,DP3-0002,2021-04-16 00:00:00,2021-12-16
2,4b81c6,DP3-0003,2021-03-29 00:00:00,2021-06-18
3,bbae56,DP3-0004,2021-03-02 00:00:00,2021-07-23
4,c6b87f,DP3-0005,2021-07-07 00:00:00,2021-08-24
...,...,...,...,...
307,2f1cc3,DP3-0429,6/14/2024,2024-07-08
308,8e5d53,DP3-0431,2024-05-13 00:00:00,2024-07-06
309,ae6c99,DP3-0431,2024-01-24 00:00:00,2024-03-20
310,c30454,DP3-0434,2024-05-31 00:00:00,2024-07-15


In [3]:
#Clean & convert to lower
log_lower = log.applymap(lambda x: x.lower() if isinstance(x, str) else x)
log = log_lower
print(sum(pd.to_datetime(log['deploy']).isna()))
log['deploy'] = pd.to_datetime(log['deploy'])
print(log['deploy'])
log['deploy'] = log['deploy'].astype('int64') // 10**9 #Convert time to unix epoch
print(log['deploy'])
print(sum(pd.to_datetime(log['return']).isna()))
log['return'] = pd.to_datetime(log['return'])
log['return'] = log['return'].astype('int64') // 10**9 #Convert time to unix epoch

log

0
0     2021-02-25
1     2021-04-16
2     2021-03-29
3     2021-03-02
4     2021-07-07
         ...    
307   2024-06-14
308   2024-05-13
309   2024-01-24
310   2024-05-31
311   2024-01-26
Name: deploy, Length: 312, dtype: datetime64[ns]
0      1614211200
1      1618531200
2      1616976000
3      1614643200
4      1625616000
          ...    
307    1718323200
308    1715558400
309    1706054400
310    1717113600
311    1706227200
Name: deploy, Length: 312, dtype: int64
0


,ID,PPN,deploy,return
0,2c564e,dp3-0001,1614211200,1623283200
1,488810,dp3-0002,1618531200,1639612800
2,4b81c6,dp3-0003,1616976000,1623974400
3,bbae56,dp3-0004,1614643200,1626998400
4,c6b87f,dp3-0005,1625616000,1629763200
...,...,...,...,...
307,2f1cc3,dp3-0429,1718323200,1720396800
308,8e5d53,dp3-0431,1715558400,1720224000
309,ae6c99,dp3-0431,1706054400,1710892800
310,c30454,dp3-0434,1717113600,1721001600


In [4]:
# Data Scrubbing
# Loop through each file in the directory, get the keys associated with that 

from networkx import is_empty


path = 'C:/Users/alexb/OneDrive/DP3/magee-pm-data-2024-09/magee-pm-data-2024-09'

for filename in os.listdir(path):
    # Check if the path is a file
    file_path = os.path.join(path, filename)
    if os.path.isfile(file_path):
        #Check ID through filename
        print(f'Processing file: {repr(filename[0:6])}')


        # Get monitor data from current file
        monitor_data = pd.read_csv(file_path)
        # set time to index for index based slicing
        monitor_data = monitor_data.set_index('unix_epoch')
        # Select only neeed readout variables
        monitor_data = monitor_data[['temperature', 'humidity', 'pressure', 'voc', 'pm_ugm3']]
        

        #Get associated ppns 
        airviz_id = filename[0:6]
        ppn_list = log.loc[log['ID']==airviz_id]
        #Init ppn frame
        ppn_save_data = None
        #Get UTC times for each ppn
        if not ppn_list.empty:
            for ppn in ppn_list['PPN']:
                
                print(ppn)
                start_time = ppn_list.loc[ppn_list['PPN'] == ppn]['deploy']
                end_time = ppn_list.loc[ppn_list['PPN'] == ppn]['return']
                if len(start_time) > 1:
                    print('looping')
                    k = 1
                    for i in start_time.index:
                        print(start_time)
                        ppn_save_data = monitor_data.loc[float(start_time[i]):float(end_time[i])]
                        save_name = '-'.join([ppn , airviz_id, str(k), '.csv'])
                        
                        
                        ppn_save_data.to_csv(''.join(['C:/Users/alexb/OneDrive/DP3/PPN - Monitor_ID - Data/', save_name]), index_label='unix_epoch' )
                        k = k+1
                else:
                    print('not looping')
                    ppn_save_data = monitor_data.loc[float(start_time):float(end_time)]
                    save_name = '-'.join([ppn , airviz_id, '.csv'])
                        
                        
                    ppn_save_data.to_csv(''.join(['C:/Users/alexb/OneDrive/DP3/PPN - Monitor_ID - Data/', save_name]), index_label='unix_epoch' )
                

Processing file: '06e5ac'
dp3-0152
not looping
dp3-0185
not looping
Processing file: '1748fa'
dp3-0008
not looping
dp3-0023
not looping
dp3-0036
not looping
dp3-0142
not looping
dp3-0174
not looping
dp3-0197
not looping
dp3-0255
not looping
dp3-0267
not looping
dp3-0308
not looping
dp3-0353
not looping
dp3-0405
not looping
dp3-0425
not looping
Processing file: '2441da'
dp3-0006
not looping
dp3-0032
not looping
dp3-0070
not looping
dp3-0123
not looping
dp3-0171
not looping
dp3-0226
not looping
dp3-0295
not looping
dp3-0330
not looping
Processing file: '268af8'
dp3-0111
not looping
dp3-0113
not looping
dp3-0160
not looping
dp3-0253
not looping
dp3-0287
not looping
dp3-0314
not looping
dp3-0357
not looping
dp3-0388
not looping
dp3-0421
not looping
Processing file: '291d69'
dp3-0034
not looping
dp3-0097
not looping
dp3-0126
not looping
dp3-0156
not looping
dp3-0169
not looping
dp3-0257
not looping
Processing file: '2c564e'
dp3-0001
not looping
dp3-0017
not looping
dp3-0024
not looping
dp3-

In [5]:
# Load in ppns where multiple monitors were given
path = 'C:/Users/alexb/OneDrive/DP3/PPN - Monitor_ID - Data'

#Init dictionary
file_groups = {}

segments = {}
#Since files are loaded by ppn sorting, we do not need to worry about holding more than one dataset at a time
# PPNs have a max of 2 monitors in sequence so a simple hold and check will work
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        # Extract file name ID 
        file_parts = filename.split('-')
        print(file_parts)


        file_id = '-'.join((file_parts[0], file_parts[1]))
        
        monitor_id = file_parts[2]
        
        key = file_id
        print(key)
        
        

        # Load the CSV into a DataFrame
        df = pd.read_csv(os.path.join(path, filename))
        
        # Check if the file ID exists in the dictionary, if not, initialize a list
        if key in file_groups:
            # Concatenate if the ID exists
            file_groups[key].append(df)
            segments[key].append(monitor_id)
            print(segments[key])
        else:
            # Create a new list for this file ID
            file_groups[key] = [df]     
            segments[key] = [monitor_id]
            print(segments[key])

['dp3', '0001', '2c564e', '.csv']
dp3-0001
['2c564e']
['dp3', '0003', '4b81c6', '.csv']
dp3-0003
['4b81c6']
['dp3', '0004', 'bbae56', '.csv']
dp3-0004
['bbae56']
['dp3', '0005', 'c6b87f', '.csv']
dp3-0005
['c6b87f']
['dp3', '0006', '2441da', '.csv']
dp3-0006
['2441da']
['dp3', '0006', 'b67138', '.csv']
dp3-0006
['2441da', 'b67138']
['dp3', '0008', '1748fa', '.csv']
dp3-0008
['1748fa']
['dp3', '0009', '4b81c6', '.csv']
dp3-0009
['4b81c6']
['dp3', '0009', 'c9fb6e', '.csv']
dp3-0009
['4b81c6', 'c9fb6e']
['dp3', '0010', 'b67138', '.csv']
dp3-0010
['b67138']
['dp3', '0011', 'a1fb71', '.csv']
dp3-0011
['a1fb71']
['dp3', '0011', 'c9fb6e', '.csv']
dp3-0011
['a1fb71', 'c9fb6e']
['dp3', '0012', '4b33be', '.csv']
dp3-0012
['4b33be']
['dp3', '0012', 'd14457', '.csv']
dp3-0012
['4b33be', 'd14457']
['dp3', '0013', 'c30454', '.csv']
dp3-0013
['c30454']
['dp3', '0015', 'c30454', '.csv']
dp3-0015
['c30454']
['dp3', '0017', '2c564e', '.csv']
dp3-0017
['2c564e']
['dp3', '0017', '8eceb4', '.csv']
dp3-0017

In [7]:
# Now, concatenate all DataFrames that share the same file ID
for key, df_list in file_groups.items():
    # Concatenate the DataFrames for each file_id
    concatenated_df = pd.concat(df_list, ignore_index=True)
    concatenated_df.sort_values(by = 'unix_epoch', ascending=True, inplace = True)

    
    filename_out = f'{key}_{segments[key][0:]}.csv'
    print(filename_out)
    
    # Save the concatenated DataFrame (optional)
    concatenated_df.to_csv(os.path.join(path, filename_out), index=False)
    
    print(f"Concatenated files for ID: {key}") 

dp3-0001_['2c564e'].csv
Concatenated files for ID: dp3-0001
dp3-0003_['4b81c6'].csv
Concatenated files for ID: dp3-0003
dp3-0004_['bbae56'].csv
Concatenated files for ID: dp3-0004
dp3-0005_['c6b87f'].csv
Concatenated files for ID: dp3-0005
dp3-0006_['2441da', 'b67138'].csv
Concatenated files for ID: dp3-0006
dp3-0008_['1748fa'].csv
Concatenated files for ID: dp3-0008
dp3-0009_['4b81c6', 'c9fb6e'].csv
Concatenated files for ID: dp3-0009
dp3-0010_['b67138'].csv
Concatenated files for ID: dp3-0010
dp3-0011_['a1fb71', 'c9fb6e'].csv
Concatenated files for ID: dp3-0011
dp3-0012_['4b33be', 'd14457'].csv
Concatenated files for ID: dp3-0012
dp3-0013_['c30454'].csv
Concatenated files for ID: dp3-0013
dp3-0015_['c30454'].csv
Concatenated files for ID: dp3-0015
dp3-0017_['2c564e', '8eceb4'].csv
Concatenated files for ID: dp3-0017
dp3-0018_['8e5d53', '8eceb4'].csv
Concatenated files for ID: dp3-0018
dp3-0019_['c6b87f'].csv
Concatenated files for ID: dp3-0019
dp3-0020_['4b33be'].csv
Concatenated fil